In [50]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [51]:
tqdm.pandas()

## Nettoyage du DF

In [52]:
# Open du fichier JSON
with open('spots_grimpe.json', 'r') as file:
    data = json.load(file)

In [53]:
# JSON TO DF
df = pd.DataFrame(data['data'])

In [54]:
# Suppression des colonnes
df = df.drop(columns= ["ref_topo", "equipement", "topo_num"])

In [55]:
# Supression du tiret situé à la fin
def sup_tiret(row):
    return row.rstrip("-")


df["type_escalade"] = df["type_escalade"].apply(sup_tiret)
df["exposition"] = df["exposition"].apply(sup_tiret)
df["public"] = df["public"].apply(sup_tiret)

In [56]:
# Création de liste à la place d'une chaîne de caractéres
def transfo_liste(chaine:str)-> list:
    liste_genres = []
    for genre in chaine.split("-"):
        liste_genres.append(genre.strip())
    return liste_genres

df["type_escalade"] = df["type_escalade"].apply(transfo_liste)
df["exposition"] = df["exposition"].apply(transfo_liste)
df["public"] = df["public"].apply(transfo_liste)
df["saison"] = df["saison"].apply(transfo_liste)

In [57]:
# Remplacer "hautniveau" par "haut niveau" dans les listes
df['public'] = df['public'].apply(lambda lst: ['haut niveau' if x == 'hautniveau' else x for x in lst])

In [58]:
# Dictionnaire de mappage
mapping_orientation = {
    "S": "Sud",
    "N": "Nord",
    "W": "Ouest",
    "E": "Est",
    "SW": "Sud-Ouest",
    "SE": "Sud-Est",
    "NW": "Nord-Ouest",
    "NE": "Nord-Est"
}

# Appliquer le mappage aux listes dans la colonne "exposition"
df["exposition"] = df["exposition"].apply(lambda lst: [mapping_orientation.get(el, el) for el in lst])

In [60]:
# Dictionnaire de mappage
mapping_mois = {
    "1": "Janvier",
    "2": "Février",
    "3": "Mars",
    "4": "Avril",
    "5": "Mai",
    "6": "Juin",
    "7": "Juillet",
    "8": "Août",
    "9": "Septembre",
    "10": "Octobre",
    "11": "Novembre",
    "12": "Decembre"

}

# Appliquer le mappage aux listes dans la colonne "saison"
df["saison"] = df["saison"].apply(lambda lst: [mapping_mois.get(el, el) for el in lst])

In [61]:
# Suppression des () et des - puis remplacer les espaces par des tirets pour l'url
df['url'] = df['nom'].str.replace('(', '').str.replace(')', '').str.replace(' - ', '-').str.replace(' ', '-')

/var/folders/fk/s9b7316s21b6krxxntv4pzzm0000gn/T/ipykernel_1485/3093565319.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['url'] = df['nom'].str.replace('(', '').str.replace(')', '').str.replace(' - ', '-').str.replace(' ', '-')


## Scrapping des urls des images

In [11]:
# Config BS4
url = "https://climbingaway.fr/fr/site-escalade/"
navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'

In [13]:
# Scrap de la class li de la page
def scrap_url_img(row):
    img_url = "https://climbingaway.fr/"
    base_url = "https://climbingaway.fr/fr/site-escalade/"
    page_url = row['url']  # Assurez-vous que cette colonne existe dans votre DataFrame
    
    # Envoie une requête pour récupérer la page HTML
    response = requests.get(base_url + page_url, headers={'User-Agent': navigator})
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Trouve la première balise <li> avec la classe 'minie_galerie_li'
    class_li = soup.find_all('li', {"class": "minie_galerie_li"})
    
    if class_li:
        # Extraire l'URL de l'image
        url_img = class_li[0].find('img').get('src')
        
        # Nettoyer l'URL en enlevant '/..' au début
        cleaned_url = url_img.lstrip('/..')
        
        # Retourner l'URL complète
        return img_url + cleaned_url
    return None  # Si aucune image n'est trouvée

df['img_url'] = df.progress_apply(scrap_url_img, axis=1)

In [1]:
import requests

In [24]:
headers = {
    "authority": "api.oblyk.org",
    "method": "GET",
    "path": "/api/v1/public/crags/873.json",
    "scheme": "https",
    "accept": "application/json, text/plain, /",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "no-cache",
    "httpapiaccesstoken": "M4rvBxc4M7kqqdtXPDvFEYm9",
    "origin": "https://oblyk.org/",
    "pragma": "no-cache",
    "priority": "u=1, i",
    "referer": "https://oblyk.org/",
    "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}


response = requests.get("https://api.oblyk.org/api/v1/public/crags/100.json", headers=headers)

In [29]:
import requests

# Définition des en-têtes
headers = {"httpapiaccesstoken": "M4rvBxc4M7kqqdtXPDvFEYm9"}


# URL de l'API
url = "https://api.oblyk.org/api/v1/public/crags/100/route_figures.json"

# Envoi de la requête GET avec les en-têtes
response = requests.get(url, headers=headers)

# Affichage de la réponse
if response.status_code == 200:
    print(response.json())  # Affiche les données JSON obtenues
else:
    print(f"Erreur {response.status_code}: {response.text}")  # Si erreur, affiche le message d'erreur


{'section_count': 12, 'route_count': 12, 'grade': {'min': {'value': 11, 'text': '2c', 'crag_route': {'id': 2715, 'name': 'Rando', 'slug_name': 'rando', 'sections_count': 1, 'grade_gap': {'max_grade_value': 11, 'min_grade_value': 11, 'max_grade_text': '2c', 'min_grade_text': '2c'}, 'crag': {'id': 100, 'name': 'Roche Beuty', 'slug_name': 'roche-beuty'}}}, 'max': {'value': 27, 'text': '5b', 'crag_route': {'id': 2711, 'name': 'Dalle', 'slug_name': 'dalle', 'sections_count': 1, 'grade_gap': {'max_grade_value': 27, 'min_grade_value': 27, 'max_grade_text': '5b', 'min_grade_text': '5b'}, 'crag': {'id': 100, 'name': 'Roche Beuty', 'slug_name': 'roche-beuty'}}}}, 'climbing_types': {'sport_climbing': 11, 'bouldering': 1, 'multi_pitch': 0, 'trad_climbing': 0, 'aid_climbing': 0, 'deep_water': 0, 'via_ferrata': 0}, 'degrees': {'1': 0, '2': 1, '3': 4, '4': 2, '5': 5, '6': 0, '7': 0, '8': 0, '9': 0}, 'levels': {'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, '2c': 1, '3a': 3, '3b': 1, '3c': 0, '4a': 0, '

In [30]:
response.json()

{'section_count': 12,
 'route_count': 12,
 'grade': {'min': {'value': 11,
   'text': '2c',
   'crag_route': {'id': 2715,
    'name': 'Rando',
    'slug_name': 'rando',
    'sections_count': 1,
    'grade_gap': {'max_grade_value': 11,
     'min_grade_value': 11,
     'max_grade_text': '2c',
     'min_grade_text': '2c'},
    'crag': {'id': 100, 'name': 'Roche Beuty', 'slug_name': 'roche-beuty'}}},
  'max': {'value': 27,
   'text': '5b',
   'crag_route': {'id': 2711,
    'name': 'Dalle',
    'slug_name': 'dalle',
    'sections_count': 1,
    'grade_gap': {'max_grade_value': 27,
     'min_grade_value': 27,
     'max_grade_text': '5b',
     'min_grade_text': '5b'},
    'crag': {'id': 100, 'name': 'Roche Beuty', 'slug_name': 'roche-beuty'}}}},
 'climbing_types': {'sport_climbing': 11,
  'bouldering': 1,
  'multi_pitch': 0,
  'trad_climbing': 0,
  'aid_climbing': 0,
  'deep_water': 0,
  'via_ferrata': 0},
 'degrees': {'1': 0,
  '2': 1,
  '3': 4,
  '4': 2,
  '5': 5,
  '6': 0,
  '7': 0,
  '8': 

In [74]:
soup = BeautifulSoup(response.content, 'html.parser')

In [75]:
soup

{"error":"You are not allowed to do this operation","code_error":"not_allowed"}

In [62]:
# Export CSV
df.to_csv("spots_grimpe.csv")

## Analyse

In [20]:
# Import CSV
df = pd.read_csv("spots_grimpe.csv")

In [71]:
df['type_escalade'].iloc[34]

['Grande voie', 'Couenne']

In [ ]:
df